# Import useful libs

In [ ]:
import numpy as np;
import pandas as pd;
import matplotlib as plt;

# Read aux CSVs

In [ ]:
anotate_codes = pd.read_csv('./datasets/anotate_codes.csv')
claim_codes = pd.read_csv('./datasets/claim_codes.csv')
commod_type_codes = pd.read_csv('./datasets/commod_type_codes.csv')
commodity_codes = pd.read_csv('./datasets/commodity_codes.csv')
concen_codes = pd.read_csv('./datasets/concen_codes.csv')
confmethod_codes = pd.read_csv('./datasets/confmethod_codes.csv')
country_codes = pd.read_csv('./datasets/country_codes.csv')
determin_codes = pd.read_csv('./datasets/determin_codes.csv')
diisttype_codes = pd.read_csv('./datasets/disttype_codes.csv')
extract_codes = pd.read_csv('./datasets/extract_codes.csv')
lab_codes = pd.read_csv('./datasets/lab_codes.csv')
mean_codes = pd.read_csv('./datasets/mean_codes.csv')
origin_codes = pd.read_csv('./datasets/origin_codes.csv')
pest_codes = pd.read_csv('./datasets/pest_codes.csv')
quantitate_codes = pd.read_csv('./datasets/quantitate_codes.csv')
state_codes = pd.read_csv('./datasets/state_codes.csv')
test_class_codes = pd.read_csv('./datasets/test_class_codes.csv')
tolerance_codes = pd.read_csv('./datasets/tolerance_codes.csv')

# Read Samples from all years

In [ ]:
samples_colums_labels = ('sample_pk','state', 'year', 'month', 'day', 'site', 'commod', 'source_id', 'variety', 'origin', 'country', 'disttype', 'commtype', 'claim', 'quantity', 'growst', 'packst', 'diistst');

In [ ]:
samples_2015 = pd.read_csv('./datasets/database/2015PDPDatabase/PDP15Samples.txt', delimiter="|", names=samples_colums_labels);
samples_2016 = pd.read_csv('./datasets/database/2016PDPDatabase/PDP16Samples.txt', delimiter="|", names=samples_colums_labels);
samples_2017 = pd.read_csv('./datasets/database/2017PDPDatabase/PDP17Samples.txt', delimiter="|", names=samples_colums_labels);
samples_2018 = pd.read_csv('./datasets/database/2018PDPDatabase/PDP18Samples.txt', delimiter="|", names=samples_colums_labels);
samples_2019 = pd.read_csv('./datasets/database/2019PDPDatabase/PDP19Samples.txt', delimiter="|", names=samples_colums_labels);

# Read Results from all years

In [ ]:
results_colums_labels = ('sample_pk','commod', 'commtype', 'lab', 'pestcode', 'testclass', 'concen', 'lod', 'conunit', 'confmethod', 'confmethod2', 'annotate', 'quantitate', 'mean', 'extract', 'determin');

In [ ]:
results_2015 = pd.read_csv('./datasets/database/2015PDPDatabase/PDP15Results.txt', delimiter="|", names=results_colums_labels);
results_2016 = pd.read_csv('./datasets/database/2016PDPDatabase/PDP16Results.txt', delimiter="|", names=results_colums_labels);
results_2017 = pd.read_csv('./datasets/database/2017PDPDatabase/PDP17Results.txt', delimiter="|", names=results_colums_labels);
results_2018 = pd.read_csv('./datasets/database/2018PDPDatabase/PDP18Results.txt', delimiter="|", names=results_colums_labels);
results_2019 = pd.read_csv('./datasets/database/2019PDPDatabase/PDP19Results.txt', delimiter="|", names=results_colums_labels);